# amyrmahdy

## Import important libraries and read data

In [ ]:
# Mount google drive
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# unzip file 
from zipfile import ZipFile

with ZipFile("/content/drive/MyDrive/Datasets/data_api_management.zip", "r") as zObject:
    zObject.extractall(path="/tmp/")


FileNotFoundError: ignored

In [ ]:
import pandas as pd

df = pd.read_parquet('https://drive.google.com/drive/folders/1EjwrzAMuYiCqUNcj65nLagCGYa1q1IZx')

ArrowInvalid: ignored

## EDA

### Overview

In [ ]:
df.shape

(1000000, 20)

In [ ]:
df.head()

,log_level,code,method,username,Price,gwStatus,planName,received_at,log_type,providerName,gatewayName,response_status,status,response_Time,paymentType,endpoint,providerResponse_Time,gwCode,call_type,api_type
0,WARNING,429,POST,username_1376,10,0,Silver,2022-06-28T10:24:46.000000Z,INPUT,provider_186,gateway_657,200,1,958,PREPAID,endpoint_1201,479,600,EXTERNAL,undefined
1,WARNING,335,GET,username_1376,10,0,Bronze,2021-09-18T05:43:21.000000Z,OUTPUT,provider_186,gateway_657,400,0,576,POSTPAID,endpoint_1201,288,404,EXTERNAL,undefined
2,WARNING,404,POST,username_1376,20,0,Gold,2022-04-03T03:52:45.000000Z,OUTPUT,provider_186,gateway_657,401,0,395,POSTPAID,endpoint_1201,197,300,INTERNAL,oauth
3,ERROR,400,POST,username_1376,20,0,Bronze,2023-02-14T18:38:42.000000Z,INPUT,provider_186,gateway_657,200,1,342,POSTPAID,endpoint_1201,171,408,INTERNAL,undefined
4,WARNING,429,GET,username_1376,50,0,Bronze,2021-07-12T11:09:06.000000Z,OUTPUT,provider_186,gateway_657,403,0,330,PREPAID,endpoint_1201,165,404,EXTERNAL,undefined


In [ ]:
df.tail()

,log_level,code,method,username,Price,gwStatus,planName,received_at,log_type,providerName,gatewayName,response_status,status,response_Time,paymentType,endpoint,providerResponse_Time,gwCode,call_type,api_type
999995,INFO,335,POST,username_114,10,0,Bronze,2022-12-09T19:11:48.000000Z,OUTPUT,provider_186,gateway_657,300,0,985,POSTPAID,endpoint_937,492,401,EXTERNAL,undefined
999996,INFO,403,POST,username_1376,10,0,Bronze,2022-11-27T23:10:03.000000Z,INPUT,provider_186,gateway_657,302,0,368,POSTPAID,endpoint_1201,184,509,EXTERNAL,undefined
999997,WARNING,404,POST,username_1376,50,1,Bronze,2022-03-31T02:07:14.000000Z,OUTPUT,provider_186,gateway_657,403,0,378,POSTPAID,endpoint_1201,189,200,INTERNAL,undefined
999998,INFO,400,POST,username_1376,10,0,Bronze,2022-12-29T17:14:25.000000Z,OUTPUT,provider_186,gateway_657,503,0,309,POSTPAID,endpoint_1201,154,403,EXTERNAL,undefined
999999,INFO,300,POST,username_1376,10,0,Gold,2022-02-20T20:11:58.000000Z,OUTPUT,provider_186,gateway_657,408,0,344,POSTPAID,endpoint_1201,172,600,INTERNAL,undefined


In [ ]:
df.describe()

,response_Time,providerResponse_Time
count,1000000.000000,1000000.000000
mean,393.562312,196.529349
std,823.455773,411.748704
min,0.000000,0.000000
25%,150.000000,75.000000
50%,249.000000,124.000000
75%,356.000000,178.000000
max,10268.000000,5134.000000


In [ ]:
df.dtypes

log_level                object
code                     object
method                   object
username                 object
Price                    object
gwStatus                 object
planName                 object
received_at              object
log_type                 object
providerName             object
gatewayName              object
response_status          object
status                   object
response_Time             int64
paymentType              object
endpoint                 object
providerResponse_Time     int64
gwCode                   object
call_type                object
api_type                 object
dtype: object

### Handle date feature

In [ ]:
# Convert object type to datetime type
df["received_at"] = pd.to_datetime(df["received_at"], utc = True)

In [ ]:
# Create some new columns from received_at column
df["received_at_short"] = df["received_at"].dt.date
df["received_at_year"] = df["received_at"].dt.year
df["received_at_month"] = df["received_at"].dt.month
df["received_at_day"] = df["received_at"].dt.day

In [ ]:
df.tail()

,log_level,code,method,username,Price,gwStatus,planName,received_at,log_type,providerName,...,paymentType,endpoint,providerResponse_Time,gwCode,call_type,api_type,received_at_short,received_at_year,received_at_month,received_at_day
999995,INFO,335,POST,username_114,10,0,Bronze,2022-12-09 19:11:48+00:00,OUTPUT,provider_186,...,POSTPAID,endpoint_937,492,401,EXTERNAL,undefined,2022-12-09,2022,12,9
999996,INFO,403,POST,username_1376,10,0,Bronze,2022-11-27 23:10:03+00:00,INPUT,provider_186,...,POSTPAID,endpoint_1201,184,509,EXTERNAL,undefined,2022-11-27,2022,11,27
999997,WARNING,404,POST,username_1376,50,1,Bronze,2022-03-31 02:07:14+00:00,OUTPUT,provider_186,...,POSTPAID,endpoint_1201,189,200,INTERNAL,undefined,2022-03-31,2022,3,31
999998,INFO,400,POST,username_1376,10,0,Bronze,2022-12-29 17:14:25+00:00,OUTPUT,provider_186,...,POSTPAID,endpoint_1201,154,403,EXTERNAL,undefined,2022-12-29,2022,12,29
999999,INFO,300,POST,username_1376,10,0,Gold,2022-02-20 20:11:58+00:00,OUTPUT,provider_186,...,POSTPAID,endpoint_1201,172,600,INTERNAL,undefined,2022-02-20,2022,2,20


## These columns can be replace with boolian values or in planName case they can encode with ordinary numbers like 1 2 3
 log_level, method, planName, log_type, paymentType, call_type, api_type 

## Visualization

In [ ]:
df_slice = df.iloc[:df.shape[0]//50, :]

In [ ]:
! pip install pygwalker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pygwalker as pyg

pyg.walk(df_slice)